In [ ]:
import json                     # module pour load le file
import re                       # module pour retourner les hasthtags
from textblob import TextBlob   # module pour analyser le sentiment du tweet
import pandas as pd             # module pour visualiser et comparer les tweets
import matplotlib.pyplot as plt

# Data utilisé pour la création des dictionnaires pour faciliter l'analyse de diagramme,...
file = open("sample2.json","r")
data =json.load(file)
file.close()

# Data utilisé pour analyser visuellement les différentes caractéristiques d'un tweet
data2 = pd.read_json("sample2.json")
df_data = pd.DataFrame(data2)

In [ ]:
class Tweet :
    def __init__(self,id_tweet,location_tweet,creation_tweet,retweet_count,tweet_language,tweet_text) :
        self.id = id_tweet
        self.location = location_tweet
        self.created = creation_tweet
        self.retweet = retweet_count
        self.language = tweet_language
        self.text = tweet_text
    
    def __str__(self):
        return f"Tweet id : {self.id}\nAuthor Location : {self.location} \nTweet Creation : {self.created} \nNumber of Retweets : {self.retweet} \nTweet Language : {self.language} \nTweet Text : {self.text}"
    
    def get_author(self):
        id_tweet = self.id
        return id_tweet
    
    def get_text(self):
        return f"The text of the tweet : {self.text}"
    
    def get_hashtags(self):
        return re.findall(r"#(\w+)", self.text)

    def get_mention(self):
        return re.findall(r"@(\w+)", self.text)
    
    def get_sentiment(self):
        text = TextBlob(self.text)
        text_polarity = text.sentiment.polarity
        if text_polarity > 0 :
            return "Positive"
        elif text_polarity < 0 :
            return "Negative"
        else :
            return "Neutral" 


#instance/ objets de la class
tweets = [Tweet(tweet["id"],tweet["AuthorLocation"],tweet["CreatedAt"],tweet["RetweetCount"],tweet["TweetLanguage"],tweet["TweetText"]) for tweet in data]

In [ ]:
#########################################################################################################################
##########################################  AVEC LES DICTIONNAIRES   ####################################################
#########################################################################################################################

# Pour Top K Hashtags -> 2 fonctions

# La première : all_hashtag() retourne tout les hashtags du fichier json dans une liste
def all_hashtag():
    l = []                                         # on intialise la liste qui va être retournée à l'éxécution de la fonction
    for tweet in tweets:                           # on parcours tout les tweets (un par un)
        if tweet.get_hashtags()== [] :             # on vérifie avec l'atribut de l'instance si la valeur retourner est  "[]" <- pas de valeur ; dans ce cas là on passe au tweet suivant
            continue
        else :
            temp = tweet.get_hashtags()            # on affecte a temp (variable temporaire) la liste des hashtags de chaque tweet
            for hashtag in temp :                  # on accède à chaque hashtag présent dans temp
                l.append(hashtag)                  # on l'affecte à la liste finale qu'on va retourner
                
    return l
   
# La deuxième : top_hashtag(k) retourne le top k (index) hashtag utilisé dans la database
def top_hashtag(k):
    hashtags = all_hashtag()                       # utilisation de la fonction all_hashtag() pour récupérer la liste complète des hashtag
    hashtag_count = {}                             # on initialise un dictionnaire vide pour compter chaque occurence de chaque hashtag
    for hashtag in hashtags :                      # ici on prend chaque hashtag un par un, ici nommé "e" (pour élément)
        if hashtag in hashtag_count :              # on vérifie si "hashtag" est dans le dictionnaire qui compte les occurences
            hashtag_count[hashtag] += 1            # dans ce cas là on incrémente sa valeur de 1
        else :
            hashtag_count[hashtag] = 1             # dans le cas contraire, on initialise la clé "hashtag" à une valeur initiale de 1 

    hashtag_count = dict(sorted(hashtag_count.items(),key = lambda x : x[1], reverse=True)) # on tri le dictionnaire en fonction des valeurs avec le lambda x[1] et reverse = True (de manière croissante)
    temp = list(hashtag_count.items())[:k]        # on converti le dictionnaire en une liste pour pouvoir effectué un slicing (afin d'obtenir les top k éléments) 
    top_k_hashtag = dict(temp)                    # on reconverti en dictionnaire afin de pouvoir manipuler les clés et valeurs facilement et dans un odre précis
    
    # Diagramme avec plt
    x_hashtag = list(top_k_hashtag.keys())        # on affecte à la variable x_hashtag les clés du dictionnaire (les hashtags (str))
    y_occurence = list(top_k_hashtag.values())    # on affecte à la variable y_occurence les valeurs des clés du dictionnaire (les occurences)

    plt.bar(x_hashtag,y_occurence)                # on créer le diagramme en bar avec en x -> x_hashtag et en y -> y_occurence
    plt.xlabel("Hashtag")                         # "Hashtag" comme titre de l'axe des x
    plt.ylabel("Occurence")                       # "Occurence" comme titre de l'axe des y

    plt.xticks(fontsize=6)                        # on précise la taille du texte pour les valeurs en x (les hashtags)
    plt.show()                                    # affichage du diagramme

    return top_k_hashtag                          # on retourne en même temps le dictionnaire trié en ordre croissant des top k hashtags

#########################################################################################################################

# Pour Top K Users -> 2 fonctions 

# La première : all_users() qui retourne tout les utilisateurs présent dans le fichier json dans une liste
def all_users():
    l = []                                       # on intialise la liste qui va être retournée à l'éxécution de la fonction
    for tweet in tweets:                         # on parcours tout les tweets (un par un)
        user = tweet.get_author()                # on extrait l'auteur grâce à la méthode .get_author() qu'on place dans la variable user
        l.append(user)                           # on ajoute à la liste l'utilisateur/id du tweet
                
    return l

# La deuxième : top_users(k) qui retourne le top k (index) d'utilisateurs/id présent dans la database
def top_users(k):
    users = all_users()                          # utilisation de la fonction all_users() pour récupérer la liste complète des utilisateurs
    user_ntweet = {}                             # on initialise un dictionnaire vide pour compter chaque occurence de chaque id
    for user in users :                          # ici on prend chaque id un par un, ici nommé "user"
        if user in user_ntweet :                 # on vérifie si l'id est déjà dans le dictionnaire
            user_ntweet[user] += 1               # si c'est le ce cas, on incrémente sa valeur de 1 
        else :
            user_ntweet[user] = 1                # sinon on crée une clé pour l'id correspondant et on initie sa valeur à 1 

    user_ntweet = dict(sorted(user_ntweet.items(),key = lambda x : x[1], reverse=True)) # on tri le dictionnaire en fonction des valeurs avec le lambda x[1] et reverse = True (de manière croissante)
    temp = list(user_ntweet.items())[:k]         # on converti le dictionnaire en une liste pour pouvoir effectué un slicing (afin d'obtenir les top k éléments)
    top_k_users = dict(temp)                     # on reconverti en dictionnaire afin de pouvoir manipuler les clés et valeurs facilement et dans un odre précis

    x_user = list(top_k_users.keys())            # on affecte à la variable x_user les clés du dictionnaire (les id (str))
    y_occurence = list(top_k_users.values())     # on affecte à la variable y_occurence les valeurs du dictionnaire (les occurences)

    plt.bar(x_user,y_occurence)                  # on créer le diagramme en bar avec en x -> x_hashtag et en y -> y_occurence
    plt.xlabel("User")                           # "User" comme titre de l'axe des x
    plt.ylabel("Occurence")                      # "Occurence" comme titre de l'axe des y

    plt.xticks(fontsize=6)                       # on précise la taille du texte pour les valeurs en x (les id)
    plt.show()                                   # affichage du diagramme


    return top_k_users                           # on retourne en même temps le dictionnaire trié en ordre croissant des top k users


#########################################################################################################################

# Pour the top K mentions -> 2 fonctions 

# La première : all_mentions() qui retourne toutes les mentions de la database
def all_mentions():
    l = []                                        # on initialise la liste qui va être utilisée pour retourner les mentions à l'éxécution de la fonction
    for tweet in tweets:
        if tweet.get_mention()== [] :             # on vérifie avec l'atribut de l'instance si la valeur retourner est  "[]" <- pas de valeur ; dans ce cas là on passe au tweet suivant
            continue
        else :
            mentions = tweet.get_mention()        # on place les mentions obtenues du tweet dans la variable temporaire mentions (qui va correspondre à une liste) 
            for mention in mentions :             # on parcours chaque élément de la liste (ils peuvent être dans une liste de liste par exemple :[[mention],mention2,...])
                l.append(mention)                 # et on l'ajoute à notre liste finale
                
    return l

def top_mention(k):
    mentions = all_mentions()                       # utilisation de la fonction all_mentions() pour récupérer la liste complète des mentions
    mention_ntweet = {}                             # on initialise un dictionnaire vide pour compter chaque occurence de chaque mention
    for mention in mentions :                       # ici on prend chaque mention, une par une
        if mention in mention_ntweet :              # on vérifie si la mention est présente dans le dictionnaire
            mention_ntweet[mention] += 1            # dans ce cas là on incrémente la valeur corresponde à la mention (la clé)
        else :
            mention_ntweet[mention] = 1             # dans le cas contraire on initie la clé correspondante à la mention à 1

    mention_ntweet = dict(sorted(mention_ntweet.items(),key = lambda x : x[1], reverse=True)) # on tri le dictionnaire en fonction des valeurs avec le lambda x[1] et reverse = True (de manière croissante)
    temp = list(mention_ntweet.items())[:k]         # on converti le dictionnaire en une liste pour pouvoir effectué un slicing (afin d'obtenir les top k éléments)
    top_k_mentions = dict(temp)                     # on reconverti la liste en dictionnaire trié en ordre croissant des top k mentions

    x_mention = list(top_k_mentions.keys())         # on affecte à la variable x_mention les clés du dictionnaire (les mentions (str))
    y_occurence = list(top_k_mentions.values())     # on affecte à la variable y_occurence les valeurs du dictionnaire (les occurences)

    plt.bar(x_mention,y_occurence)                  # on créer le diagramme en bar avec en x -> x_mention et en y -> y_occurence
    plt.xlabel("Mention",)                          # on créer le diagramme en bar avec en x -> x_mention
    plt.ylabel("Occurence")                         # on créer le diagramme en bar avec en y -> y_occurence

    plt.xticks(fontsize=7)                          # on précise la taille du texte pour les valeurs en x (les mentions)
    plt.show()                                      # affichage du diagramme

    return top_k_mentions                           # on retourne en même temps le dictionnaire trié en ordre croissant des top k mentions

#########################################################################################################################

def top_topics():
    pass


#########################################################################################################################
##########################################  AVEC LES DATAFRAMES   #######################################################
#########################################################################################################################


def all_tweet_mention(mention):                                                               # fonction qui retourne l'ensemble des tweets mentionnant un utilisateur spécifique dans un dataframe
    return df_data[df_data["TweetText"].str.contains(f"@{mention}",regex=False)]              # on tri les lignes de la database pour extraire seulement celle qui contiennent @mention dans la colonne 

def all_tweet_hashtag(hashtag):                                                               # fonction qui retourne l'ensemble des tweets faisant référence à un hashtag spécifique
    return df_data[df_data["TweetText"].str.contains(f"#{hashtag}",regex=False)]              # on tri les lignes de la database pour extraire seulement celle qui contiennent #hashtag dans la colonne de TweetText

def user_specified_hashtag(hashtag):                                                          # fonction qui retourne les id de chaque utilisateur mentionnant un hashtag spécifique
    return df_data[df_data["TweetText"].str.contains(f"#{hashtag}",regex=True)].get("id")     # on tri les lignes de la database pour extraire  les #hashtag présent dans la colonne TweetText et on extrait seulement les id

def user_mentionned(user):                                                                    # fonction qui retourne les utilisateurs mentionnés par un utilisateur spécifique
    text = str(df_data[df_data["id"]==user].get("TweetText"))                                 # on attribue à la variable text le texte du tweet qui correspond à l'utilisiateur spécifique
    return re.findall(r"@(\w+)", text)                                                        # on nettoie le text en extrayant les mentions avec le r"@(\w+)"



In [ ]:
user_mentionned(1417013867725656064)

In [ ]:
all_tweet_hashtag("MachineLearning")

In [ ]:
all_tweet_mention("nigewillson")

In [ ]:
user_mentionned(1415291886860967936)

In [ ]:
df_data

In [ ]:
top_hashtag(5)

In [ ]:
all_tweet_mention("NEJM")

In [ ]:
# Test for Top k hashtags
all_hashtag()
top_hashtag(3)

In [ ]:
# Test for Top k users
all_users()
top_users(4)

In [ ]:
# Test for Top k mentions
all_mentions()
top_mention(7)

In [ ]:
all_tweet_hashtag("DataScience")

In [ ]:
all_tweet_mention("SpirosMargaris")

In [ ]:
df_data

In [ ]:
all_hashtag()

In [ ]:
user_specified_hashtag("FEATURED")

In [ ]:
user_mentionned(1415291886860967936)

In [ ]:
all_tweet_hashtag("machine")

In [ ]:
str = "My day is shit"


In [ ]:
tweets[0].get()

In [ ]:
text = "bonjouir comment ca va "
a = TextBlob(text)
print(a.polarity)
if a.polarity > 0 :
    print("Positive")
elif a.polarity < 0 :
    print( "Negative")
else :
    print( "Neutral" )

In [ ]:
def nettoyage_tweet(text):
  # Pattern trouvé sur le lien suivant : https://gist.github.com/Alex-Just/e86110836f3f93fe7932290526529cd1#gistcomment-3208085
  
  pattern = re.compile(
    "(["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "])"
  )
  text = re.sub(pattern, r'', text)                  # ici l'expression regulière va supprimer (sub en anglais) les émojis et caractères contenus dans le pattern
  return text

nettoyage_tweet("Je m'appelle ahmad j'ai 45 ans 🏢, mais je suis francais haha 🤣")

On considère InPoDA, une plateforme pour l’analyse de données de réseaux sociaux. InPoDa 
est une plateforme fictive. Voici donc des hypothèses personnelles sur son fonctionnement :  
Lorsqu'un  utilisateur  rédige  une  publication  (un  tweet),  elle  est  publiée  à  travers  un  script 
python qui vérifie d’abord la structure de l’objet tweet, le nettoie en éliminant les caractères 
spéciaux  (J,  etc.)  au  cas  où  on  en  a ,  et  le  stockera  dans  un  fichier,  appelé  «  zone 
d’atterrissage ». En pièces jointes un jeu de publications (tweets) que vous pouvez utiliser. 
N’hésitez pas à utiliser un autre jeu de tweets si vous le pensez nécessaire.

In [ ]:
def create_tweet():
    tweet_data = {
        "id": int(input("Please enter your id: ")),
        "AuthorLocation": input("Enter your location: "),
        "CreatedAt": input("Enter the date of creation of your tweet: "),
        "RetweetCount": int(input("How much retweet does your tweet have? ")),
        "TweetLanguage": input("Please enter your language (ex 'en','fr'): "),
        "TweetText": input("Enter the text of your tweet: ")
    }

    # Charger le contenu actuel de la base de données depuis le fichier JSON
    try:
        with open("filetest.json", "r") as file:
            database = json.load(file)
    except FileNotFoundError:
         # Si le fichier n'existe pas, initialisez la base de données comme une liste vide
         database = []

    # Ajouter le nouveau tweet à la liste de dictionnaires
    database.append(tweet_data)

    # Enregistrer la liste mise à jour dans le fichier JSON
    with open("filetest.json", "w") as file:
        json.dump(database, file)

create_tweet()